# `JsonProvider`

The introductory JSON parser of F# is in ` FSharp.Data` [[GitHub](https://github.com/fsprojects/FSharp.Data)], the `JsonProvider` [[GitHub](https://github.com/fsprojects/FSharp.Data/blob/main/src/Json/JsonProvider.fs)]:

In [ ]:
#!fsharp

#r "nuget: FSharp.Data"

Installed Packages FSharp.Data, 4.2.7

In [ ]:
#!fsharp

open FSharp.Data

type OneProvider = JsonProvider<""" { "name":"John", "age":94 } """>


In [ ]:
#!fsharp

let oneDoc = OneProvider.Parse(""" { "name":"Tomas", "age":4 } """)

oneDoc.GetType()

FSharp.Data.Runtime.BaseTypes.JsonDocument

We see that `oneDoc` is of type `JsonDocument` [📖 [docs](https://fsprojects.github.io/FSharp.Data/reference/fsharp-data-runtime-basetypes-jsondocument.html)] _but_ this is different from `System.Text.Json.JsonDocument` [📖 [docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsondocument?view=net-6.0)]. We can actually “dot into” `oneDoc` to see the JSON properties of our domain data:

In [ ]:
#!fsharp

oneDoc.Name |> printfn "%s"
oneDoc.Age |> printfn "%i"

Tomas
4


The code for these properties, `Name` and `Age`, were effectively _generated_ by the F# compiler! This technology is similar to _T4 templates_ [📖 [docs](https://docs.microsoft.com/en-us/visualstudio/modeling/code-generation-and-t4-text-templates?view=vs-2022)] but much more.

## can `JsonProvider` infer from JSON objects as properties?

Can `JsonProvider` work with JSON properties that are objects?

In [ ]:
#!fsharp

[<Literal>]
let SampleJson = """
    {
        "firstName": "John",
        "lastName": "Henry",
        "address": {
            "street": "One Zuck Zuck Way",
            "city": "Siliconburg",
            "state": "CA",
            "zip": 90045
        }
    }
"""

type TwoProvider = JsonProvider<SampleJson>

We see that the answer is _yes_!

In [ ]:
#!fsharp

let twoDoc = TwoProvider.Parse(SampleJson)

twoDoc.GetType()

FSharp.Data.Runtime.BaseTypes.JsonDocument

We also see that `twoDoc` is the same CLR type as `oneDoc` which is the first hint that working with `JsonProvider` while using types to enforce business rules could be in conflict.

In [ ]:
#!fsharp

twoDoc.FirstName |> printfn "%s"
twoDoc.LastName |> printfn "%s"

John
Henry


We clearly see that `JsonProvider` is automatically camel-casing the properties it is auto-generating. The property `Address` is regarded as just another F# `JsonDocument`:

In [ ]:
#!fsharp

twoDoc.Address |> printfn "%A"
twoDoc.Address.GetType()

{
  "street": "One Zuck Zuck Way",
  "city": "Siliconburg",
  "state": "CA",
  "zip": 90045
}


FSharp.Data.Runtime.BaseTypes.JsonDocument

`JsonProvider` automatically infers an integer type for the `Address.Zip` property:

In [ ]:
#!fsharp

twoDoc.Address.Zip |> printfn "%i"
twoDoc.Address.Zip.GetType() = typeof<int>

90045


True

Because `Address` is just an F# `JsonDocument`, the `JsonDocument,JsonValue.Properties()` sequence can be piped into `dict`:

In [ ]:
#!fsharp

twoDoc.Address.JsonValue.Properties() |> dict

key,type,Item,_Print
street,FSharp.Data.JsonValue+String,One Zuck Zuck Way,"""One Zuck Zuck Way"""
city,FSharp.Data.JsonValue+String,Siliconburg,"""Siliconburg"""
state,FSharp.Data.JsonValue+String,CA,"""CA"""
zip,FSharp.Data.JsonValue+Number,90045,90045


Let’s zoom out and see the whole document:

In [ ]:
#!fsharp

twoDoc.JsonValue.Properties() |> dict

key,type,Item,_Print,properties
firstName,FSharp.Data.JsonValue+String,John,"""John""",
lastName,FSharp.Data.JsonValue+String,Henry,"""Henry""",
address,FSharp.Data.JsonValue+Record,,"{ ""street"": ""One Zuck Zuck Way"", ""city"": ""Siliconburg"", ""state"": ""CA"", ""zip"": 90045 }","[ ( street, { ""One Zuck Zuck Way"": Item: One Zuck Zuck Way, _Print: ""One Zuck Zuck Way"" } ), ( city, { ""Siliconburg"": Item: Siliconburg, _Print: ""Siliconburg"" } ), ( state, { ""CA"": Item: CA, _Print: ""CA"" } ), ( zip, { 90045: Item: 90045, _Print: 90045 } ) ]"


The output above is denoting that the row with key `address` is type `FSharp.Data.JsonValue+Record`. This `+Record` type is, again, not some kind of reference to native F# records. This `+Record` type is part of the `JsonValue` discriminated union [[GitHub](https://github.com/fsprojects/FSharp.Data/blob/main/src/Json/JsonValue.fs#L38)], defined specifically for use with `JsonConverter`.

## transforming an F# record into a `JsonProvider` `JsonDocument`

A [StackOverflow answer](https://stackoverflow.com/a/57054259/22944) leads us along a particular path:

In [ ]:
#!fsharp

let jsonRecord = JsonValue.Record [||]
jsonRecord.ToString()

{
}

The binding to `jsonRecord` above defines an empty array, `[||]`, of `(string * JsonValue)` tuples. At the very least, the string output of `jsonRecord` can be used to generate sample JSON for `JsonProvider` initialization.

In [ ]:
#!fsharp

let jsonRecord = JsonValue.Record [|
    "street", JsonValue.String "One Zuck Zuck Way"
    "city", JsonValue.String "Siliconburg"
    "state", JsonValue.String "CA"
    "zip", JsonValue.Number 90045m
|]

jsonRecord.ToString()

{
  "street": "One Zuck Zuck Way",
  "city": "Siliconburg",
  "state": "CA",
  "zip": 90045
}

@[BryanWilhite](https://twitter.com/BryanWilhite)
